In [97]:
import pandas as pd
import yaml
import numpy as np
import random
from math import log2, sqrt, exp


In [115]:
# setting of the parameters of neural networks
L = 4; # Nro of the layers in the neural networks
x = np.array([0,1,5,3,2]) # inputs
N_l= np.array([x.size,3,3,2],dtype=int) # array that content the number of the neurons for each layers
weights= {} # save the weights of the neural network
# we generate the weights randomly
for i in range(0,L-1):
    weights[i] = np.random.rand(N_l[i+1],N_l[i]+1)
alpha = 1

In [116]:
def g(theta):
    return 1/(1+np.exp(-theta))

def Update_weight(weights,d_weights,aplha):
    for i in range(0,L-1):
        weights[i] = weights[i] - aplha*d_weights
    return weights

def forward(x,L,N_l,weights):
    for i in range(0,L-1):
        z = weights[i].dot(np.append(x,1))
        x = g(z)
    return x